In [1]:
import pandas as pd
import plotly.express as pe

from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split


In [2]:
path = r"C:\Users\harsh\Desktop\NPCI-Python-ML\datasets\defaulters_data.csv"

df = pd.read_csv (path, index_col="clientid")

df

income        age         loan  default
clientid                                               
1         66155.925095  59.017015  8106.532131        0
2         34415.153966  48.117153  6564.745018        0
3         57317.170063  63.108049  8020.953296        0
4         42709.534201  45.751972  6103.642260        0
5         66952.688845  18.584336  8770.099235        1
...                ...        ...          ...      ...
1996      59221.044874  48.518179  1926.729397        0
1997      69516.127573  23.162104  3503.176156        0
1998      44311.449262  28.017167  5522.786693        1
1999      43756.056605  63.971796  1622.722598        0
2000      69436.579552  56.152617  7378.833599        0

[2000 rows x 4 columns]

### step 2: Data exploration

In [3]:
df.shape

(2000, 4)

In [4]:
df.index

Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
            ...
            1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000],
           dtype='int64', name='clientid', length=2000)

In [5]:
df.columns

Index(['income', 'age', 'loan', 'default'], dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 1 to 2000
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   income   2000 non-null   float64
 1   age      1997 non-null   float64
 2   loan     2000 non-null   float64
 3   default  2000 non-null   int64  
dtypes: float64(3), int64(1)
memory usage: 78.1 KB


In [7]:
df.isna().sum()

income     0
age        3
loan       0
default    0
dtype: int64

In [8]:
df.nunique()

income     2000
age        1997
loan       2000
default       2
dtype: int64

In [9]:
df["default"].value_counts()

0    1717
1     283
Name: default, dtype: int64

### features

In [10]:
real_value_features = ["loan", "age", "income"]

In [11]:
df[     real_value_features     ].describe()

loan          age        income
count   2000.000000  1997.000000   2000.000000
mean    4444.369695    40.807559  45331.600018
std     3045.410024    13.624469  14326.327119
min        1.377630   -52.423280  20014.489470
25%     1939.708847    28.990415  32796.459717
50%     3974.719419    41.317159  45789.117313
75%     6432.410625    52.587040  57791.281668
max    13766.051239    63.971796  69995.685578

## confirm outliers

In [12]:
for col in real_value_features :
    fig = pe.box(   y=col, data_frame=df  )
    display(fig)

### step 2b) Preprocessing the data!

In [13]:
condition = (df["age"] >= 0)

data = df[    condition    ].copy() #makes new table "data"

del df #delete reference to original table


data.dropna(inplace=True) #drop missing values. DO THIS INPLACE meaning in the same object!


PYTHON


n1 = 10
n2 = 20
n3 = 20


                           10  <----------- n1

              n3---------> 20 <--------------n2 



In [14]:
display(data[["age"]].describe())


display(data.info())


age
count  1994.000000
mean     40.927700
std      13.271802
min      18.055189
25%      29.043284
50%      41.382673
75%      52.616900
max      63.971796

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 1 to 2000
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   income   1994 non-null   float64
 1   age      1994 non-null   float64
 2   loan     1994 non-null   float64
 3   default  1994 non-null   int64  
dtypes: float64(3), int64(1)
memory usage: 77.9 KB


None

# PCA


In [15]:
for count in [1,2]:
    pca_model = PCA(     n_components=count   ) 
    pca_model.fit_transform(    data[real_value_features]    )

    print(f"explained variance ratio: {pca_model.explained_variance_ratio_}"     )

explained variance ratio: [0.96546129]
explained variance ratio: [0.96546129 0.03453789]


In [16]:
final_pca_output = PCA(n_components=1).fit_transform(  data[real_value_features] )
pca_data = pd.DataFrame(final_pca_output,   columns=["PC1"])
pca_data

PC1
0    -21083.963806
1     10656.901448
2    -12278.478387
3      2446.182314
4    -21941.254923
...            ...
1989 -13583.121502
1990 -23982.491697
1991    908.059818
1992   1838.598417
1993 -24278.708946

[1994 rows x 1 columns]

### HOW DOES THE NEW FEATURE SEPARATE MY DATA?

In [17]:
pe.box(
    y="PC1",
    data_frame=pca_data
)

In [18]:

pe.strip(      
    x=data["default"],
    y=pca_data["PC1"],
    color=data["default"]
)

### step 3: separate features & labels

In [19]:
features = ["PC1"]

target = "default"

## step 4: split the data into training and testing sets

## x refers to features
## y refers to target



features for training
features for testing

target for training (corresponding to features for training)
target for testing (corresponding to features for testing)

In [20]:
x_train, x_test, y_train, y_test = train_test_split(   
    pca_data[features], #feature
    data[target],  #target
    test_size=0.2 ,  #size for testing
    random_state=10, #random state
    stratify=data[target]  #ratio for stratification
) #

### step 5: training the model

In [21]:
model = SVC( kernel="poly", degree=4  )

In [22]:
model.fit(    x_train, y_train        )

SVC(degree=4, kernel='poly')

### step 6: evaluation of model

In [25]:
ans = model.predict(     x_test    ) #testing features 

print( classification_report(    y_test,  ans     ) )

              precision    recall  f1-score   support

           0       0.86      1.00      0.92       342
           1       0.00      0.00      0.00        57

    accuracy                           0.86       399
   macro avg       0.43      0.50      0.46       399
weighted avg       0.73      0.86      0.79       399



c:\Users\harsh\Desktop\NPCI-Python-ML\virtual_setup\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\harsh\Desktop\NPCI-Python-ML\virtual_setup\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\harsh\Desktop\NPCI-Python-ML\virtual_setup\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [24]:
actual_ans_df = y_test.reset_index(drop=True)

predicted_ans_df = pd.DataFrame(ans)

result_df = pd.concat(    [actual_ans_df,  predicted_ans_df], axis=1     )

result_df.columns = [    "Actual_Answer", "Predicted_ans"    ]

result_df

Actual_Answer  Predicted_ans
0                0              0
1                0              0
2                0              0
3                0              0
4                0              0
..             ...            ...
394              0              0
395              0              0
396              0              0
397              0              0
398              0              0

[399 rows x 2 columns]